
# マルコフ決定プロセス (MDP)

 :label: `sec_mdp`このセクションでは、マルコフ決定プロセス (MDP) を使用して強化学習問題を定式化する方法について説明し、MDP のさまざまなコンポーネントについて詳しく説明します。

##  MDPの定義

マルコフ決定プロセス (MDP) :cite: `BellmanMDP`さまざまなアクションがシステムに適用されるときにシステムの状態がどのように変化するかを示すモデルです。いくつかの異なる量が集まって MDP を形成します。 

![](../img/mdp.png) :幅: `250px` :ラベル: `fig_mdp`
-  $\mathcal{S}$ を MDP の状態のセットとする。具体的な例として、グリッドワールドをナビゲートするロボットの :numref: `fig_mdp`を参照してください。この場合、$\mathcal{S}$ は、任意のタイムステップでロボットが存在できる一連の位置に対応します。
-  $\mathcal{A}$ を、ロボットが各状態で実行できる一連のアクション (たとえば、「前に進む」、「右に曲がる」、「左に曲がる」、「同じ場所に留まる」など) とします。ロボットの現在の状態を、セット $\mathcal{S}$ 内の他の状態に変更できます。
- ロボットが*どの*ように動くのか正確には分からないが、ある程度の近似値までしか分からない場合があります。この状況を強化学習で次のようにモデル化します。ロボットが「前に進む」アクションをとった場合、低い確率で現在の状態に留まり、別の低い確率で「左に曲がる」などになります。数学的には、これは次のようになります。これは、$T(s, a, s&#39;) となるような「遷移関数」 $T: \mathcal{S} \times \mathcal{A} \times \mathcal{S} \to [0,1]$ を定義することに相当します。 = P(s&#39; \mid s, a)$ は、ロボットが状態 $s$ にあり、アクション $a$ を実行したとして、状態 $s&#39;$ に到達する条件付き確率を使用します。遷移関数は確率分布であるため、すべての $s \in \mathcal{S}$ に対して $\sum_{s&#39; \in \mathcal{S}} T(s, a, s&#39;) = 1$ となります。 $a \in \mathcal{A}$、つまり、ロボットがアクションを実行する場合、ロボットは何らかの状態に移行する必要があります。
- ここで、どのアクションが有用でどのアクションが「報酬」の概念を使用していないのかという概念を構築します $r: \mathcal{S} \times \mathcal{A} \to \mathbb{R}$。ロボットが状態 $s$ でアクション $a$ を実行すると、ロボットは報酬 $r(s,a)$ を得ると言います。報酬 $r(s, a)$ が大きい場合、状態 $s$ でアクション $a$ を取る方が、ロボットの目標、つまり温室に行くことを達成するのに役立つことを示します。報酬 $r(s, a)$ が小さい場合、アクション $a$ はこの目標を達成するのにあまり役に立ちません。報酬はユーザー (強化学習アルゴリズムを作成する人) が目標を念頭に置いて設計することに注意することが重要です。

## 返品と割引率

上記のさまざまなコンポーネントが一緒になってマルコフ決定プロセス (MDP) を形成します $$\text{MDP}: (\mathcal{S}, \mathcal{A}, T, r).$$

ここで、ロボットが特定の状態 $s_0 \in \mathcal{S}$ で開始し、軌道 $$\tau = (s_0, a_0, r_0, s_1, a_1, r_1, s_2、a_2、r_2、\ldots).$$

各タイム ステップ $t$ で、ロボットは状態 $s_t$ にあり、報酬 $r_t = r(s_t, a_t)$ をもたらすアクション $a_t$ を実行します。軌道の*リターンは*、そのような軌道に沿ってロボットによって得られる報酬の合計です $$R(\tau) = r_0 + r_1 + r_2 + \cdots.$$

強化学習の目標は、最大の*リターン*を持つ軌道を見つけることです。

ロボットがゴール位置に到達することなくグリッドワールド内を移動し続ける状況を考えてください。この場合、軌跡内の状態とアクションのシーケンスは無限に長くなり、そのような無限に長い軌跡の*戻りは*無限になります。このような軌道に対しても強化学習の定式化を意味のあるものにしておくために、割引係数 $\gamma &lt; 1$ の概念を導入します。割引*リターン*は $$R(\tau) = r_0 + \gamma r_1 + \gamma^2 r_2 + \cdots = \sum_{t=0}^\infty \gamma^t r_t.$$ と書きます。

 $\gamma$ が非常に小さい場合、遠い将来 ($t = 1000$ など) にロボットが獲得する報酬は、係数 $\gamma^{1000}$ によって大幅に割り引かれることに注意してください。これにより、ロボットはその目標、つまり、gridwold の例では温室に行くという目標を達成する短い軌道を選択するようになります (:numref: `fig_mdp`を参照)。割引係数の値が大きい場合 ($\gamma = 0.99$ など)、ロボットは*探索し*、目標位置に向かう最適な軌道を見つけることが奨励されます。

## マルコフ仮定の議論

新しいロボットについて考えてみましょう。状態 $s_t$ は上記の位置ですが、アクション $a_t$ は、「前進」のような抽象的なコマンドではなく、ロボットが車輪に加える加速度です。このロボットが状態 $s_t$ でゼロ以外の速度を持っている場合、次の位置 $s_{t+1}$ は過去の位置 $s_t$、加速度 $a_t$、およびロボットの速度の関数になります。時刻 $t$ は $s_t - s_{t-1}$ に比例します。これは、次のようにする必要があることを示しています。

 $$s_{t+1} = \text{何らかの関数}(s_t, a_t, s_{t-1});$$

この場合の「ある関数」はニュートンの運動法則です。これは、単純に $s_t$ と $a_t$ に依存する遷移関数とはまったく異なります。

マルコフ系とは、次の状態 $s_{t+1}$ が現在の状態 $s_t$ と現在の状態で取られるアクション $a_t$ の関数にすぎないすべてのシステムです。マルコフ システムでは、次の状態は、過去にどのようなアクションが取られたか、またはロボットが過去にいた状態には依存しません。たとえば、上記のアクションとして加速度を持つ新しいロボットは、次の位置 $s_{t+1}$ が速度を通じて前の状態 $s_{t-1}$ に依存するため、マルコフ関数ではありません。システムのマルコフ的性質は限定的な仮定であるように見えるかもしれませんが、そうではありません。マルコフ決定プロセスは依然として、非常に大きなクラスの実際のシステムをモデル化することができます。たとえば、新しいロボットの場合、状態 $s_t$ をタプル $(\text{location}, \text{velocity})$ に選択した場合、次の状態 $(\text{location} であるため、システムはマルコフ行列になります) *{t+1}, \text{velocity}* {t+1})$ は現在の状態 $(\text{location}_t, \text{velocity}_t)$ と現在の状態 $a_t でのアクションにのみ依存します$。

## まとめ

強化学習問題は通常、マルコフ決定プロセスを使用してモデル化されます。マルコフ決定プロセス (MDP) は、4 つのエンティティ $(\mathcal{S}, \mathcal{A}, T, r)$ のタプルによって定義されます。ここで、$\mathcal{S}$ は状態空間、$\mathcal {A}$ はアクション空間、$T$ は MDP の遷移確率をエンコードする遷移関数、$r$ は特定の状態でアクションをとることによって得られる即時報酬です。

## 演習
1. [MountainCar](https://www.gymlibrary.dev/environments/classic_control/mountain_car/)問題をモデル化する MDP を設計するとします。<ol><li>状態の集合は何になるでしょうか?
1. 一連のアクションは何でしょうか?
1. 考えられる報酬関数は何でしょうか?



[ディスカッション](https://discuss.d2l.ai/t/12084)
